## Увод

Ова лекција ће обухватити:
- Шта је позивање функција и у којим ситуацијама се користи
- Како направити позив функције користећи OpenAI
- Како интегрисати позив функције у апликацију

## Циљеви учења

Након што завршите ову лекцију, знаћете и разумећете:

- Сврху коришћења позивања функција
- Како подесити позив функције користећи OpenAI сервис
- Како осмислити ефикасне позиве функција за потребе ваше апликације


## Разумевање позивања функција

У овој лекцији желимо да направимо функционалност за нашу едукативну стартап платформу која омогућава корисницима да преко чет-бота пронађу техничке курсеве. Препоручићемо курсеве који одговарају њиховом нивоу знања, тренутној улози и технологији која их занима.

Да бисмо ово реализовали, користићемо комбинацију:
 - `OpenAI` за креирање чет искуства за корисника
 - `Microsoft Learn Catalog API` да помогнемо корисницима да пронађу курсеве на основу њихових захтева
 - `Function Calling` да преузме кориснички упит и пошаље га функцији која ће направити API захтев

За почетак, хајде да видимо зашто бисмо уопште желели да користимо позивање функција:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # добијамо нови одговор од GPT који може да види одговор функције


print(second_response.choices[0].message)


### Зашто коришћење функција

Ако сте завршили било коју другу лекцију у овом курсу, вероватно већ разумете колико су Моћни Велики Језички Модели (LLMs). Надамо се да сте приметили и нека њихова ограничења.

Функционално позивање (Function Calling) је могућност у OpenAI сервису која је направљена да реши следеће изазове:

Неконзистентно форматирање одговора:
- Пре увођења функционалног позивања, одговори великог језичког модела били су неструктурирани и непоуздани. Програмери су морали да пишу сложен код за валидацију како би обрадили све варијације у излазу.

Ограничена интеграција са спољним подацима:
- Пре ове могућности, било је тешко укључити податке из других делова апликације у контекст ћаскања.

Стандартизовањем формата одговора и омогућавањем лаке интеграције са спољним подацима, функционално позивање поједностављује развој и смањује потребу за додатном логиком за валидацију.

Корисници нису могли да добију одговоре попут „Какво је тренутно време у Стокхолму?“. Ово је зато што су модели били ограничени на период када су подаци били обучавани.

Погледајмо пример испод који илуструје овај проблем:

Рецимо да желимо да направимо базу података о студентима како бисмо им препоручили одговарајући курс. Испод имамо два описа студената који су веома слични по подацима које садрже.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Желимо да пошаљемо ово LLM-у ради анализе података. Ово се касније може користити у нашој апликацији за слање ка API-ју или чување у бази података.

Хајде да направимо два идентична упутства којима ћемо објаснити LLM-у које информације нас занимају:


Желимо да ово пошаљемо LLM-у да би анализирао делове који су важни за наш производ. Тако можемо направити два идентична упутства да дамо LLM-у:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Након креирања ова два упита, послаћемо их LLM-у користећи `openai.ChatCompletion`. Упит чувамо у променљивој `messages` и додељујемо улогу `user`. Ово је да бисмо имитирали поруку коју корисник шаље чет-боту.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Сада можемо послати оба захтева LLM-у и испитати одговор који добијемо.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Иако су упити исти и описи слични, можемо добити различите формате својства `Grades`.

Ако покренете горњу ћелију више пута, формат може бити `3.7` или `3.7 GPA`.

Ово се дешава зато што LLM прима неструктуриране податке у облику написаног упита и такође враћа неструктуриране податке. Потребан нам је структуриран формат како бисмо знали шта да очекујемо када чувамо или користимо те податке.

Коришћењем функционалног позивања можемо обезбедити да добијамо структуиране податке. Када користимо функционално позивање, LLM заправо не позива и не извршава никакве функције. Уместо тога, креирамо структуру коју LLM треба да прати у својим одговорима. Затим користимо те структуиране одговоре да бисмо знали коју функцију да покренемо у нашим апликацијама.


![Дијаграм тока позива функције](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.sr.png)


### Примери употребе позива функција

**Позивање спољашњих алата**  
Ћаскање са ботовима је одлично за добијање одговора на питања корисника. Кроз позивање функција, ботови могу да користе поруке корисника како би обавили одређене задатке. На пример, студент може да затражи од бота: „Пошаљи имејл мом професору и реци да ми је потребна додатна помоћ око ове теме“. Ово може да покрене позив функције `send_email(to: string, body: string)`

**Креирање API или база података упита**  
Корисници могу да пронађу информације користећи природни језик који се затим претвара у форматиран упит или API захтев. Пример за ово је наставник који пита: „Који су ученици завршили последњи задатак“, што може да покрене функцију `get_completed(student_name: string, assignment: int, current_status: string)`

**Креирање структурираног података**  
Корисници могу да узму блок текста или CSV и користе LLM да извуку важне информације из њега. На пример, студент може да претвори Википедијин чланак о мировним споразумима у AI флеш картице. Ово се може урадити коришћењем функције `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Креирање вашег првог позива функције

Процес креирања позива функције обухвата три главна корака:
1. Позивање Chat Completions API-ја са списком ваших функција и поруком корисника
2. Читање одговора модела ради извршавања неке радње, односно покретања функције или API позива
3. Поновно позивање Chat Completions API-ја са одговором ваше функције како бисте ту информацију искористили за креирање одговора кориснику.


![Ток функцијског позива](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.sr.png)


### Елементи позива функције

#### Кориснички унос

Први корак је да се креира порука корисника. Ово може бити динамички додељено узимањем вредности из текстуалног уноса или можете овде доделити вредност. Ако први пут радите са Chat Completions API-јем, потребно је да дефинишемо `role` и `content` поруке.

`role` може бити `system` (правила), `assistant` (модел) или `user` (крајњи корисник). За позивање функције, ово ћемо поставити као `user` и дати пример питања.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Креирање функција.

Следеће ћемо дефинисати функцију и њене параметре. Овде ћемо користити само једну функцију под називом `search_courses`, али можете направити више функција.

**Важно**: Функције се укључују у системску поруку LLM-у и улазе у укупан број доступних токена које имате на располагању.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Дефиниције**

Структура дефиниције функције има више нивоа, при чему сваки има своје особине. Ево прегледа угнежђене структуре:

**Особине функције на највишем нивоу:**

`name` - Име функције коју желимо да позовемо.

`description` - Опис начина на који функција ради. Овде је важно бити прецизан и јасан.

`parameters` - Листа вредности и формата које желите да модел врати у свом одговору.

**Особине објекта параметара:**

`type` - Тип података за објекат параметара (обично "object")

`properties` - Листа конкретних вредности које ће модел користити у свом одговору

**Особине појединачног параметра:**

`name` - Имплицитно дефинисано кључем својства (нпр. "role", "product", "level")

`type` - Тип података за овај конкретан параметар (нпр. "string", "number", "boolean")

`description` - Опис конкретног параметра

**Опционе особине:**

`required` - Низ који наводи који су параметри неопходни да би се позив функције успешно извршио


### Позив функције
Након што смо дефинисали функцију, сада треба да је укључимо у позив ка Chat Completion API-ју. То радимо тако што додамо `functions` у захтев. У овом случају то је `functions=functions`.

Постоји и опција да се подеси `function_call` на `auto`. То значи да ћемо дозволити LLM-у да сам одлучи коју функцију треба позвати на основу поруке корисника, уместо да то ми ручно одређујемо.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Сада да погледамо одговор и видимо како је форматиран:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Можете да видите да је име функције позвано и да је LLM из корисничке поруке успео да пронађе податке који одговарају аргументима функције.


## 3.Интеграција позива функција у апликацију.

Након што смо тестирали форматиран одговор од LLM-а, сада можемо да га интегришемо у апликацију.

### Управљање током

Да бисмо ово интегрисали у нашу апликацију, хајде да следимо ове кораке:

Прво, позваћемо OpenAI сервисе и сачувати поруку у променљиву под именом `response_message`.


In [ ]:
response_message = response.choices[0].message

Сада ћемо дефинисати функцију која ће позвати Microsoft Learn API да добије листу курсева:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Као најбољу праксу, прво ћемо проверити да ли модел жели да позове неку функцију. Након тога, креираћемо једну од доступних функција и повезаћемо је са функцијом која се позива.  
Затим ћемо узети аргументе функције и мапирати их на аргументе из LLM-а.

На крају, додаћемо поруку о позиву функције и вредности које су враћене поруком `search_courses`. Ово даје LLM-у све потребне информације да  
кориснику одговори природним језиком.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Изазов са кодом

Одличан посао! Да бисте наставили са учењем о OpenAI Function Calling, можете направити: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
- Још параметара функције који могу помоћи полазницима да пронађу више курсева. Доступне параметре API-ја можете пронаћи овде:  
- Направите још један позив функције који узима више информација од полазника, као што је њихов матерњи језик  
- Додајте обраду грешака када позив функције и/или API-ја не врати ниједан одговарајући курс



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
